# Import modelflow

In [11]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

from ipywidgets import interact,Dropdown,Checkbox,Tab
from IPython.display import display, clear_output,Latex, Markdown
import pandas as pd

import modeljupyter as mj
from modelvis import waterplot
from modeljupyter import jupviz



from modelclass import model
model.widescreen()
model.scroll_off()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:

from IPython.display import display

# Load model and data, and run 

In [3]:
if not 'masia' in locals():
    print('Load model')
    masia,baseline = model.modelload('Asia.pcim',run=0,silent=1)
from inject import  inject
inject(masia)

Load model


In [4]:
xx = masia(baseline,stats=0,progressbar=1)

Solving Asia:   0%|          | 0/30

In [5]:
co2_input_var = [var for var in masia.totgraph.predecessors('WLD_CO2') if var.endswith('CO2')]

In [6]:
co2_baseline = baseline.loc[:,co2_input_var].pipe(lambda df:df.rename(columns={c : masia.iso_dict.get(cname := c.split('_')[0],cname) for c in df.columns}))

# Variables to inputwidget

In [7]:
outvar = 'GDI GCARBR CO2 LNN'.split()
displayvar = 'WLD_CO2 ASP_CO2 G20_CO2 '

#  masia.countries_GCARBR_A is list of all countries in the model with a countries_GCARBR_A 
#  masia.countries_ASEAN is list of all countries in ASEAN 
countries_rest = [country for country in masia.countries_GCARBR_A if country not in masia.countries_ASEAN]

slidedef_allasean =  {'All ASEAN countries' : {'var': ' '.join([f'{country}_GCARBR_A' for country in masia.countries_ASEAN ])
                          , 'value': 0.0,'min':0,'max':100,'op':'+impulse','step':10,'dec':0}}
slidedef_allrest =  {'Rest of the world' : {'var': 
    ' '.join([f'{country}_GCARBR_A' for country in countries_rest])
                          , 'value': 0.0,'min':0,'max':100,'op':'+impulse','step':10,'dec':0}}
slidedef_countries = {masia.iso_dict.get(country,country)+', Extra' : 
                      {'var': f'{country}_GCARBR_A', 'value': 0.0,'min':-40,'max':40,'op':'+impulse','step':10,'dec':5} 
                      for country in  masia.countries_ASEAN}



In [8]:
updatevar = [f'{country}_GCARBR_A' for country in masia.countries_ASEAN ]
smalldf = baseline.loc[2022:2027, updatevar]

In [9]:
%%html
<style>
.jupyter-widgets.widget-tab > .p-TabBar .p-TabBar-tab {
    flex: 0 1 auto
}
</style>

In [21]:
from modelwidget import slidewidget, tabwidget, sheetwidget, updatewidget

inject(masia)
tabdef = {'Areawide Carbon tax 2022'  : slidewidget( {**slidedef_allasean,**slidedef_allrest}),
          'Single country add on 2022': slidewidget(slidedef_countries),
          'Individual tax rates'      : (sheet1 :=  sheetwidget(df_var = smalldf,transpose=0,trans=masia.get_a_country))
         }


wtab = tabwidget(tabdefdict = tabdef,tab=False)

wtabtotal = tabwidget(tabdefdict = {'Test11':wtab},
                                   
                                    tab=True,selected_index = 0)

xx = updatewidget(masia,wtabtotal,lwreset=True,varpat = displayvar,outputwidget='keep_viz')

xx.wtotal

Output()

In [25]:
!edit  ~/.jupyter/nbconfig/notebook.json


'edit' is not recognized as an internal or external command,
operable program or batch file.
